In [1]:
import os, sys

parent = os.path.abspath('..')
sys.path.insert(1, parent)

In [2]:
from helper_funcs.preprocessing import total_timeseries, get_covariates
from helper_funcs.inverse import inverse_func
from helper_funcs.error import error_count
from helper_funcs.prediction import model_compare

from darts.utils.model_selection import train_test_split
from darts.dataprocessing.transformers import StaticCovariatesTransformer, Scaler
from pytorch_lightning.callbacks import ModelCheckpoint, RichProgressBar
from darts.models import NLinearModel, TFTModel
from darts import TimeSeries

import pandas as pd 

RANDOM = 101
INPUT_CHUNK = 15
OUTPUT_CHUNK = 5
TEST_SIZE = 0.2
RETRAIN=True
LAST=False
RESET=False
EXP_MA = 15
SAVE = True
EPOCHS = 100

timeseries = total_timeseries(INPUT_CHUNK, market=True, sentiment=True, embeddings=True)
timeseries = StaticCovariatesTransformer().fit_transform(timeseries)

train, val = train_test_split(
    timeseries,
    axis=1,
    test_size=TEST_SIZE,
    input_size=INPUT_CHUNK,
    horizon=OUTPUT_CHUNK,
    vertical_split_type='model-aware'
)

data = [train, val]

target_train, past_train, future_train, target_val, past_val, future_val, target_test, past_test, future_test = get_covariates(
    data,
    target='Close',
    past_covariates=[],
    embeddings=False
    )

scaler_target = Scaler()
scaler_covariates = Scaler()

target_train_scaled = scaler_target.fit_transform(target_train)
target_val_scaled = scaler_target.transform(target_val)
target_test_scaled = scaler_target.transform(target_test)

In [3]:
# names = ['model_nlinear_HLOV', 'model_nlinear_HLOVS1', 'model_nlinear_HLOVS2', 'model_nlinear_HLOVE', 'model_tft_HLOV', 'model_tft_HLOVS1', 'model_tft_HLOVS2', 'model_tft_HLOVE']
names = ['model_baseline_ ', 'model_nlinear_HLOV', 'model_nlinear_HLOVS2', 'model_nlinear_HLOVE', 'model_tft_HLOV', 'model_tft_HLOVS1', 'model_tft_HLOVE']

In [4]:
aapl = model_compare(scaler_target, target_val_scaled, names, 0, (20, 300))

                Rank      MAPE       MAE        R2      RMSE     SMAPE
baseline_          1  2.565375  1.313199  0.934381  1.673613  2.586731
nlinear_HLOV       1  2.550596  1.303323  0.927964  1.707390  2.548938
nlinear_HLOVS2     1  2.684359  1.362238  0.925333  1.738284  2.681404
nlinear_HLOVE      1  4.476497  2.269002  0.804964  2.809401  4.462753
tft_HLOV           1  2.469620  1.258324  0.938346  1.579567  2.480466
tft_HLOVS          1  2.498124  1.277238  0.929188  1.692819  2.489215
tft_HLOVE          1  2.148479  1.092153  0.950150  1.420327  2.146826


In [5]:
amzn = model_compare(scaler_target, target_val_scaled, names, 1, (20, 300))

                Rank      MAPE       MAE        R2      RMSE     SMAPE
baseline_          1  1.483209  0.790689  0.723525  1.027324  1.483695
nlinear_HLOV       1  1.698444  0.904396  0.619211  1.213525  1.700342
nlinear_HLOVS2     1  1.661883  0.886539  0.640279  1.179476  1.665045
nlinear_HLOVE      1  3.716309  1.976295 -0.606492  2.492559  3.714378
tft_HLOV           1  1.378455  0.735566  0.762233  0.958919  1.378033
tft_HLOVS          1  1.761824  0.929144  0.598621  1.245901  1.748397
tft_HLOVE          1  1.616580  0.859356  0.664042  1.139853  1.601282


In [6]:
goog = model_compare(scaler_target, target_val_scaled, names, 2, (20, 300))

                Rank      MAPE       MAE        R2      RMSE     SMAPE
baseline_          1  2.293928  1.231937  0.711881  1.608714  2.294996
nlinear_HLOV       1  2.841432  1.525060  0.479005  2.134403  2.841179
nlinear_HLOVS2     1  2.826060  1.512826  0.470252  2.152259  2.824009
nlinear_HLOVE      1  5.357076  2.903965 -0.516098  3.641022  5.365030
tft_HLOV           1  2.076677  1.116324  0.747884  1.484774  2.078060
tft_HLOVS          1  1.877557  1.006942  0.768733  1.422056  1.870129
tft_HLOVE          1  2.065734  1.102186  0.708829  1.595637  2.048597


In [7]:
googl = model_compare(scaler_target, target_val_scaled, names, 3, (20, 300))

                Rank      MAPE       MAE        R2      RMSE     SMAPE
baseline_          1  2.265531  1.202937  0.703121  1.576262  2.265684
nlinear_HLOV       1  2.791058  1.482187  0.475684  2.067612  2.789953
nlinear_HLOVS2     1  2.788387  1.479178  0.484328  2.050498  2.786320
nlinear_HLOVE      1  6.955437  3.728700 -1.913053  4.873564  7.003946
tft_HLOV           1  1.999451  1.062851  0.751383  1.423763  1.998928
tft_HLOVS          1  1.844931  0.978699  0.763876  1.387529  1.837526
tft_HLOVE          1  2.028139  1.069502  0.705733  1.548970  2.009747


In [8]:
msft = model_compare(scaler_target, target_val_scaled, names, 4, (20, 300))

                Rank      MAPE       MAE        R2      RMSE     SMAPE
baseline_          1  1.815929  1.182108  0.930340  1.477347  1.828686
nlinear_HLOV       1  2.002080  1.302072  0.911860  1.642360  2.003518
nlinear_HLOVS2     1  2.070136  1.344079  0.907226  1.684984  2.071514
nlinear_HLOVE      1  3.733400  2.417324  0.677150  3.143277  3.734783
tft_HLOV           1  1.739176  1.144219  0.929364  1.470261  1.754619
tft_HLOVS          1  1.681465  1.108834  0.931103  1.452057  1.693740
tft_HLOVE          1  1.693565  1.096888  0.935928  1.400289  1.681869


In [9]:
tsla = model_compare(scaler_target, target_val_scaled, names, 5, (10, 300))

                Rank       MAPE       MAE        R2      RMSE     SMAPE
baseline_          1   5.102709  2.032091  0.799570  2.729829  5.091106
nlinear_HLOV       1   5.967695  2.419837  0.710660  3.233697  5.980887
nlinear_HLOVS2     1   5.976965  2.436333  0.708964  3.243160  5.977145
nlinear_HLOVE      1   6.484025  2.565531  0.696476  3.312009  6.480512
tft_HLOV           1   5.249876  2.113657  0.746230  3.028414  5.182109
tft_HLOVS          1  10.611443  4.472159 -0.280298  6.802219  9.784446
tft_HLOVE          1   5.182447  2.070537  0.783993  2.794018  5.135710


In [10]:
# pd.concat({
#     'Apple': aapl,
#     'Amazon' : amzn,
#     'Google': goog,
#     'Microsoft': msft,
#     'Tesla' : tsla,
#     }, axis=0).round(decimals=4)

In [11]:
# print(pd.concat({
#     'Apple': aapl,
#     'Amazon' : amzn,
#     'Google': goog,
#     'Microsoft': msft,
#     'Tesla' : tsla,
#     }, axis=0).round(decimals=4).to_latex())